In [114]:
import numpy as np
np.set_printoptions(suppress=True, precision=5)

In [115]:
from numpy import inf

Q = np.array([[inf, 1,   5,   3,   inf, inf, inf],
              [inf, inf, inf, 9,   6,   inf, inf],
              [inf, inf, inf, inf, inf, 2,   inf],
              [inf, inf, inf, inf, inf, 4,   8],
              [inf, inf, inf, inf, inf, inf, 4],
              [inf, inf, inf, inf, inf, inf, 1],
              [inf, inf, inf, inf, inf, inf, 0]])

In [116]:
nodes = range(7) # number the nodes from 0-6
J = np.zeros_like(nodes, dtype = int) # our first guess
next_J = np.empty_like(nodes, dtype = int) # updated guesses

max_iter = 500
i = 0
while i < max_iter:
    for v in nodes:
        # finding the min over all choices of w, hence Q[v,:]
        next_J[v] = np.min(Q[v,:]+J)
    # convergence criteria
    if np.array_equal(next_J, J):
        break

    J[:] = next_J # conclude this iteration by updating J with our current guess
    i += 1

print("The cost-to-go function is", J)

The cost-to-go function is [ 8 10  3  5  4  1  0]


# Exercise 38.1
The text in `graph.txt` below describes a weighted directed graph.

The line node0, node1 0.04, node8 11.11, node14 72.21 means that from node0 we can go to

    node1 at cost 0.04

    node8 at cost 11.11

    node14 at cost 72.21

No other nodes can be reached directly from node0.

Other lines have a similar interpretation.

Your task is to use the algorithm given above to find the optimal path and its cost.

In [117]:
graph = open("graph.txt", "r")
contents = graph.read()
graph.close()

rows = contents.strip().split("\n")
data = [row.split(", ") for row in rows]
print(data)

[['node0', 'node1 0.04', 'node8 11.11', 'node14 72.21'], ['node1', 'node46 1247.25', 'node6 20.59', 'node13 64.94'], ['node2', 'node66 54.18', 'node31 166.80', 'node45 1561.45'], ['node3', 'node20 133.65', 'node6 2.06', 'node11 42.43'], ['node4', 'node75 3706.67', 'node5 0.73', 'node7 1.02'], ['node5', 'node45 1382.97', 'node7 3.33', 'node11 34.54'], ['node6', 'node31 63.17', 'node9 0.72', 'node10 13.10'], ['node7', 'node50 478.14', 'node9 3.15', 'node10 5.85'], ['node8', 'node69 577.91', 'node11 7.45', 'node12 3.18'], ['node9', 'node70 2454.28', 'node13 4.42', 'node20 16.53'], ['node10', 'node89 5352.79', 'node12 1.87', 'node16 25.16'], ['node11', 'node94 4961.32', 'node18 37.55', 'node20 65.08'], ['node12', 'node84 3914.62', 'node24 34.32', 'node28 170.04'], ['node13', 'node60 2135.95', 'node38 236.33', 'node40 475.33'], ['node14', 'node67 1878.96', 'node16 2.70', 'node24 38.65'], ['node15', 'node91 3597.11', 'node17 1.01', 'node18 2.57'], ['node16', 'node36 392.92', 'node19 3.49', '

In [118]:
Q = np.full((100,100), inf)
nodes = range(100)
for i in nodes: # each row in graph.txt, each i is the index of the row, not the row itself
    for j in range(1,len(data[i])): # each element in each row
        info = data[i][j].split() # splits node1 0.04 into ['node1', '0.04']
        weight = float(info[1])
        node_num = int(info[0][4:]) # node is a 4 letter word, so skip the first 4 and then take everything
        Q[i,node_num] = weight # update Q[row, column] = respective weight

Q[99,99] = 0 # cost to destination is zero
print(Q)

[[ inf 0.04  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 [ inf  inf  inf ...  inf  inf  inf]
 ...
 [ inf  inf  inf ...  inf 0.3   inf]
 [ inf  inf  inf ...  inf  inf 0.33]
 [ inf  inf  inf ...  inf  inf 0.  ]]


In [119]:
J = np.zeros_like(nodes, dtype = float)
next_J = np.empty_like(nodes, dtype = float)

max_iter = 500
i = 1

while i < max_iter:
    for v in nodes:
        next_J[v] = np.min(Q[v,:]+J) # bellman eqn

    if np.array_equal(next_J, J):
        break

    J[:] = next_J
    i+=1

print(J)

[160.55 162.26  88.52 143.73 145.12 147.43 141.67 144.1  149.44 140.95
 150.8  141.99 148.93 303.77 130.85 107.01 128.15 114.66 104.44 124.66
 124.42 168.62 200.27  88.21 114.61 102.74 112.81 112.8  131.97  70.38
  71.45 176.51  66.16  65.84 110.18  64.7  156.07  67.8   67.44  63.95
  77.15  62.61  58.66 149.25  50.72  52.26  67.53  48.58  65.21  46.27
  45.76  54.36 135.03  44.38  54.99  42.16  40.05  40.03  62.47  30.69
  33.02  37.5   35.56  38.77  32.62  34.98  34.34  31.39  31.68  30.47
  30.41  30.02  35.96  22.04  21.16  21.45  20.64  42.31  79.71   8.91
  33.37  77.12  15.27  10.37  33.5    7.46  85.72   4.8    4.59  37.6
  13.56  22.8   11.87   3.28   3.09   0.27   1.06   0.63   0.33   0.  ]


In [120]:
path = []

def bellman(v):
    if v == len(nodes)-1: # we hit node99
        path.append(v)
        return path

    neighbors = []
    for w in range(len(Q[v,:])):
        if Q[v,:][w] != inf: # if an edge exists between v and w
            neighbors.append(w) # they are neighbors

    min_val = inf
    next_node = 0
    for w in neighbors:
        cur_val = Q[v,w] + J[w] # bellman
        if cur_val < min_val:
            min_val = cur_val
            next_node = w

    path.append(v) # we only visit vertices along the path, so we only append them
    bellman(next_node)

bellman(0) # finding optimal path from node0 to node99
print(path)

[0, 8, 11, 18, 23, 33, 41, 53, 56, 57, 60, 67, 70, 73, 76, 85, 87, 88, 93, 94, 96, 97, 98, 99]


In [121]:
cost = 0
for node in range(1,len(path)):
    cost += Q[path[node-1], path[node]] # cost from one vertex to the next along our path
print(cost)

160.55000000000007


In [122]:
print(f"The optimal path is {path}")
print(f"The cost to travel along it is {round(cost,3)}")
print(J[0]) # cost matches with J[0]

The optimal path is [0, 8, 11, 18, 23, 33, 41, 53, 56, 57, 60, 67, 70, 73, 76, 85, 87, 88, 93, 94, 96, 97, 98, 99]
The cost to travel along it is 160.55
160.55
